In [1]:
import pandas as pd
import re
import os
from IPython.display import display

# ===== 0) 파일 경로 =====
input_file  = "/Users/mac/Documents/SORA_Project/data/raw/Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv"
output_dir  = "/Users/mac/Documents/SORA_Project/data/raw/preprocessing"
os.makedirs(output_dir, exist_ok=True)  # 폴더가 없으면 자동 생성

output_file = os.path.join(output_dir, "Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM_전처리.csv")

# ===== 1) 데이터 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
print(f"✅ 원본 데이터 로드 완료: {df.shape}")
print("\n📌 데이터 기본 정보:")
df.info()


✅ 원본 데이터 로드 완료: (2045, 20)

📌 데이터 기본 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      2045 non-null   object 
 1   시도청     2024 non-null   object 
 2   경찰서     2024 non-null   object 
 3   관서명     2024 non-null   object 
 4   구분      2024 non-null   object 
 5   전화번호    2024 non-null   object 
 6   주소      2024 non-null   object 
 7   입력주소    2024 non-null   object 
 8   X       2024 non-null   float64
 9   Y       2024 non-null   float64
 10  CLSS    2024 non-null   object 
 11  PNU     2022 non-null   float64
 12  주소구분    2024 non-null   object 
 13  표준신주소   2010 non-null   object 
 14  표준구주소   2024 non-null   object 
 15  우편번호    1960 non-null   float64
 16  행정동코드   1997 non-null   float64
 17  행정동명    1997 non-null   object 
 18  법정동코드   1990 non-null   float64
 19  법정동명    1997 non-null   object 
dtypes: float64(6), object(14)
memory 

In [2]:
# ===== 2) 좌표만 숫자형 변환 후 자릿수 고정 =====
coord_cols = ["위도", "경도", "X좌표", "Y좌표"]
for col in coord_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

def format_coords(x, decimals):
    if pd.isna(x):
        return None
    try:
        return round(float(x), decimals)
    except:
        return x

if "위도" in df.columns:
    df["위도"] = df["위도"].apply(lambda x: format_coords(x, 6))
if "경도" in df.columns:
    df["경도"] = df["경도"].apply(lambda x: format_coords(x, 6))
if "X좌표" in df.columns:
    df["X좌표"] = df["X좌표"].apply(lambda x: format_coords(x, 4))
if "Y좌표" in df.columns:
    df["Y좌표"] = df["Y좌표"].apply(lambda x: format_coords(x, 5))

# ===== 3) 코드형 컬럼 문자열 정리 =====
code_cols = ["PNU", "행정동코드", "법정동코드", "시도코드", "시군구코드", "읍면동코드", "우편번호"]
def clean_code_str(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    s = re.sub(r"[^\d]", "", s)  # 숫자만 남기기
    return s if s else None

for c in code_cols:
    if c in df.columns:
        df[c] = df[c].apply(clean_code_str)

# ===== 4) 주소 파싱 (표준신주소 → 표준구주소) =====
def parse_address(addr):
    if pd.isna(addr) or str(addr).strip() == "":
        return None, None, None, None
    parts = re.split(r"\s+", str(addr).strip())
    si   = parts[0] if len(parts) > 0 else None
    gu   = parts[1] if len(parts) > 1 and re.search(r"(시|군|구)$", parts[1]) else None
    dong = parts[2] if len(parts) > 2 else None
    road = " ".join(parts[3:]) if len(parts) > 3 else None
    return si, gu, dong, road

def pick_standard_address(row):
    if "표준신주소" in row and pd.notna(row["표준신주소"]) and str(row["표준신주소"]).strip() != "":
        return row["표준신주소"]
    if "표준구주소" in row and pd.notna(row["표준구주소"]) and str(row["표준구주소"]).strip() != "":
        return row["표준구주소"]
    return None

df["기준주소"] = df.apply(pick_standard_address, axis=1)
df[["시", "구", "동", "도로명"]] = df["기준주소"].apply(lambda x: pd.Series(parse_address(x)))

# ===== 5) NaN 보완 (도로명주소·지번주소 활용) =====
def fill_address_from_row(row):
    # 도로명주소
    if pd.notna(row.get("도로명주소")):
        parts = str(row["도로명주소"]).strip().split()
        if len(parts) >= 3:
            row["시"] = row["시"] or parts[0]
            row["구"] = row["구"] or parts[1]
            row["동"] = row["동"] or parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])
    # 지번주소
    if pd.notna(row.get("지번주소")):
        parts = str(row["지번주소"]).strip().split()
        if len(parts) >= 3:
            row["시"] = row["시"] or parts[0]
            row["구"] = row["구"] or parts[1]
            row["동"] = row["동"] or parts[2]
            if pd.isna(row["도로명"]) and len(parts) > 3:
                row["도로명"] = " ".join(parts[3:])
    return row

df = df.apply(fill_address_from_row, axis=1)

# ===== 6) 좌표 기반 보완 (있을 때만) =====
coord_lookup_cols = []
if "위도" in df.columns and "경도" in df.columns:
    coord_lookup_cols = ["위도", "경도"]
elif "X좌표" in df.columns and "Y좌표" in df.columns:
    coord_lookup_cols = ["X좌표", "Y좌표"]

if coord_lookup_cols:
    coord_lookup = df.dropna(subset=["시", "구", "동", "도로명"]).set_index(coord_lookup_cols)[["시", "구", "동", "도로명"]]

    def fill_from_coords(row):
        if any(pd.isna(row[c]) for c in ["시", "구", "동", "도로명"]) and tuple(row[coord_lookup_cols]) in coord_lookup.index:
            vals = coord_lookup.loc[tuple(row[coord_lookup_cols])]
            for c in ["시", "구", "동", "도로명"]:
                if pd.isna(row[c]):
                    row[c] = vals[c]
        return row

    df = df.apply(fill_from_coords, axis=1)
else:
    print("⚠️ 좌표 컬럼이 없어 좌표 기반 보완은 건너뜀")

# ===== 7) 컬럼 순서 정렬 =====
target_cols = ["시", "구", "동", "도로명"]
cols = list(df.columns)
if "유형명" in cols:
    type_idx = cols.index("유형명")
else:
    type_idx = 0

cols = [c for c in cols if c not in target_cols]
new_cols = cols[:type_idx+1] + target_cols + cols[type_idx+1:]
df = df[new_cols]

print("\n✅ 시/구/동/도로명 컬럼 이동 완료")
print(df.columns.tolist())

# ===== 8) 저장 =====
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n💾 전처리 완료 CSV 저장: {output_file}")

# ===== 9) 결과 확인 =====
print("\n🧾 결과 컬럼:")
print(df.columns.tolist())
display(df.head(10))

⚠️ 좌표 컬럼이 없어 좌표 기반 보완은 건너뜀

✅ 시/구/동/도로명 컬럼 이동 완료
['연번', '시', '구', '동', '도로명', '시도청', '경찰서', '관서명', '구분', '전화번호', '주소', '입력주소', 'X', 'Y', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명', '기준주소']

💾 전처리 완료 CSV 저장: /Users/mac/Documents/SORA_Project/data/raw/preprocessing/Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM_전처리.csv

🧾 결과 컬럼:
['연번', '시', '구', '동', '도로명', '시도청', '경찰서', '관서명', '구분', '전화번호', '주소', '입력주소', 'X', 'Y', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명', '기준주소']


,연번,시,구,동,도로명,시도청,경찰서,관서명,구분,전화번호,...,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명,기준주소
0,1,서울특별시,중구,퇴계로49길,13,서울청,서울중부,을지,지구대,02-2279-1908,...,1114013300100200118,새주소,서울특별시 중구 퇴계로49길 13,서울특별시 중구 충무로5가 20-43,45590,11140590000,광희동,11140133000,충무로5가,서울특별시 중구 퇴계로49길 13
1,2,서울특별시,중구,퇴계로,375-1,서울청,서울중부,광희,지구대,02-2233-1444,...,1114016200102518,새주소,서울특별시 중구 퇴계로 375-1,서울특별시 중구 신당동 250-10,45660,11140615000,신당동,11140162000,신당동,서울특별시 중구 퇴계로 375-1
2,3,서울특별시,중구,동호로5길,15,서울청,서울중부,약수,지구대,02-2234-8112,...,11140162001084418,새주소,서울특별시 중구 동호로5길 15,서울특별시 중구 신당동 844-3,45960,11140635000,약수동,11140162000,신당동,서울특별시 중구 동호로5길 15
3,4\t서울청\t서울중부\t신당\t파출소\t02-2252-0435\t 서울특별시 중구...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,...,None,NaN,NaN,NaN,None,None,NaN,None,NaN,None
4,5,서울특별시,중구,동호로,261,서울청,서울중부,장충,파출소,02-2274-9003,...,1114014400101960118,새주소,서울특별시 중구 동호로 261,서울특별시 중구 장충동2가 196-1,46200,11140580000,장충동,11140144000,장충동2가,서울특별시 중구 동호로 261
5,6,서울특별시,중구,퇴계로,178,서울청,서울중부,충무,파출소,02-2278-7710,...,1114013700100200118,새주소,서울특별시 중구 퇴계로 178,서울특별시 중구 필동1가 20-5,46270,11140570000,필동,11140137000,필동1가,서울특별시 중구 퇴계로 178
6,7,서울특별시,중구,충무로,56-1,서울청,서울중부,을지로3가,파출소,02-2266-2404,...,1114015500102910118,새주소,서울특별시 중구 충무로 56-1,서울특별시 중구 을지로3가 291-50,45490,11140605000,을지로동,11140155000,을지로3가,서울특별시 중구 충무로 56-1
7,8,서울특별시,종로구,종로17길,4,서울청,서울종로,종로2가,지구대,02-3701-4301,...,11110138001003918,새주소,서울특별시 종로구 종로17길 4,서울특별시 종로구 종로2가 39-3,31400,11110615000,종로1.2.3.4가동,11110138000,종로2가,서울특별시 종로구 종로17길 4
8,9,서울특별시,종로구,삼일대로15길,19,서울청,서울종로,관수,파출소,02-3701-4302,...,1111013500100110118,새주소,서울특별시 종로구 삼일대로15길 19,서울특별시 종로구 관철동 11-19,31900,11110615000,종로1.2.3.4가동,11110135000,관철동,서울특별시 종로구 삼일대로15길 19
9,10,서울특별시,종로구,세검정로,226,서울청,서울종로,세검정,파출소,02-3701-4507,...,11110185001009518,새주소,서울특별시 종로구 세검정로 226,서울특별시 종로구 홍지동 95-1,30170,11110550000,부암동,11110185000,홍지동,서울특별시 종로구 세검정로 226
